In [2]:
import pandas as pd
import pyodbc
private_key = 'pamf-dwh-775beaae50ff.json'
project_id = 'pamf-dwh'


read_local = pd.read_excel('Recouvrement prêt délinquants du 07-12-19 au 15-12-19.xlsx',skiprows=6)
read_local = read_local[['Alias','Msisdn']]
#creating rcontactID
read_local['rcontactID'] = read_local.Alias.str.extract('(\d+)')
read_local.dropna(inplace=True)
read_local['rcontactID'] = read_local['rcontactID'].apply(lambda x: int(x))
read_local['Msisdn'] = read_local['Msisdn'].apply(lambda x: '0'+str(int(x)))
read_local.rename(columns = {"rcontactID": "rContactID"}, inplace=True)

##############################################
sql = """ SELECT * FROM `pamf-dwh.DFS_VIEW.numero_clients`  """
read_gbq = pd.read_gbq(sql,project_id = project_id ,private_key = private_key, dialect= 'standard')
read_gbq.rename(columns = {"rContactID": "rContactID", "Msisdn":"Msisdn", "Alias":"Alias"}, inplace=True)


#############################################
to_gbq = read_gbq.append(read_local)
to_gbq.drop_duplicates(inplace=True)

to_gbq.to_gbq('DFS_VIEW.numero_clients',project_id=project_id,if_exists ='replace',private_key= private_key)

E:\Anaconda3\lib\site-packages\pandas\io\gbq.py:149: FutureWarning: private_key is deprecated and will be removed in a future version.Use the credentials argument instead. See https://pandas-gbq.readthedocs.io/en/latest/howto/authentication.html for examples on using the credentials argument with service account keys.
  credentials=credentials, verbose=verbose, private_key=private_key)
E:\Anaconda3\lib\site-packages\pandas\io\gbq.py:162: FutureWarning: private_key is deprecated and will be removed in a future version.Use the credentials argument instead. See https://pandas-gbq.readthedocs.io/en/latest/howto/authentication.html for examples on using the credentials argument with service account keys.
  credentials=credentials, verbose=verbose, private_key=private_key)
1it [00:28, 28.19s/it]
